**Homework**

* **1bp** Analyze the *Harry Potter* eye/smile detector and it's failcases. Come with possible causes for this issue, based on the way variation is encoded and used in searching for matches. Propose a solution for the identified *eye detector* failcase.
    * The Harry Potter eye detector fails due to the fact that Harry is wearing glasses. Glasses are  partially
transparent, this means that some light is reflected (the refraction could also be a negative factor, for high diopter lenses).
Moreover, the margin of the glasses have pixels with a different intensity, changing the
variance around the eyes, which might trick the algorithm.
    * For smile, it is similar, the classifier learns to recognize features similar to two semicircles with some variation
of the color in the center. More training instances could be used such that the classifiers also learns the correct range of
colors.
    * A solution would be to apply histogram equalization before the haar-cascade.
* **1bp** To overcome the difficulty of MeanShift when the object orientation was changing dramatically, the CamShift algorithm was proposed. Experiment with *OpenCV* CamShift on a video sequence where MeanShift performs poorly.

In [4]:
import cv2 as cv
import numpy as np

video_capture = cv.VideoCapture("sample_video.avi")
out_dir = 'meanshift_demo'


ret_cap, frame = video_capture.read()
print(frame.shape)

# initialize window for face
x, y, w, h = (100, 45, 25, 30)
track_window = (x, y, w, h)

# set the region of interest (ROI)
roi = frame[y: y+h, x: x+w]
roi_hsv = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(roi_hsv, np.array((0., 60.,32.)), np.array((180.,255.,255.)))

roi_hist = cv.calcHist([roi_hsv],[0],mask,[180],[0,180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)



# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )


fnums = []
x_s = []
y_s = []
ret = True
fnum = 0

while True:
  ret_cap, frame = video_capture.read()
  fnum += 1

  if not ret_cap:
    break

  hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
  dst = cv.calcBackProject([hsv], [0], roi_hist, [0,180], 1) # find the best batching displacement
  ret, track_window = cv.CamShift(dst, track_window, term_crit) # use meanshift based on the displacement
  x,y,w,h = track_window

  plot_img = frame.copy()

  x_s.append(x + w // 2)
  y_s.append(y + h // 2)
  cv.rectangle(plot_img, (x, y), (x+w, y+h), (255, 255, 255), 3)
  fnums.append(fnum)

  cv.imwrite("{}/1-{}.png".format(out_dir, str(fnum - 1).zfill(3)), plot_img)


(120, 160, 3)


* **1bp** Given a stereo-vision system and an obstacle detection problem, describe a pipeline used to decide if your moving object fits through the space between two identified obstacles. (No implementation needed. Describe how would you approach the given problem, based on the topics discussed in the exercise sessions).
    * The two identified obstacles should be tracked. The distance between their margin is traced at each frame (frame 0, frame 1, ...)
 and based on the speed of the moving object and the displacement of the pixels from 1 frame to another we are able to calculate
the distance between the object and each obstacle (using Triangle Similarity) and the relative distance between obstacles.